In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import os
import cv2
import numpy as np
import pandas as pd
#!pip install ultralytics
from ultralytics import YOLO

import matplotlib.pyplot as plt
import torch


In [7]:
model_pred = YOLO("D:\BITS\PS1\pose\posenet-python-master\yolov8x-pose.pt")

In [8]:
def reshape_to_square(image):
    height, width = image.shape[:2]
    min_dimension = min(height, width)
    x = int((width - min_dimension) / 2)
    y = int((height - min_dimension) / 2)
    cropped_image = image[y:y + min_dimension, x:x + min_dimension]
    resized_image = cv2.resize(cropped_image, (640, 640))
    return resized_image

def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image from {image_path}")
        return None
    resized_image = (image)
    predictions = model_pred.predict(resized_image)
    return predictions

def extract_keypoints(predictions):
    keypoints = []
    for result in predictions:
        if result.keypoints is not None:
            keypoints.append(result.keypoints.xyn.cpu().numpy().tolist())
    return keypoints

def process_dataset(dataset_path):
    data = []
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith((".jpg", ".jpeg", ".png")):
                file_path = os.path.join(root, file)
                predictions = process_image(file_path)
                if predictions:
                    keypoints = extract_keypoints(predictions)
                    label = os.path.basename(root)
                    data.append(keypoints)
    return data

In [36]:
train_data = process_dataset(r"D:\archith\DATASET\TRAIN")




0: 448x640 1 person, 1826.4ms
Speed: 3577.9ms preprocess, 1826.4ms inference, 836.3ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 220.5ms
Speed: 2.0ms preprocess, 220.5ms inference, 7.6ms postprocess per image at shape (1, 3, 448, 640)

0: 480x640 1 person, 255.2ms
Speed: 3.9ms preprocess, 255.2ms inference, 5.9ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 2 persons, 223.3ms
Speed: 9.8ms preprocess, 223.3ms inference, 12.5ms postprocess per image at shape (1, 3, 448, 640)

0: 320x640 1 person, 265.8ms
Speed: 2.0ms preprocess, 265.8ms inference, 4.1ms postprocess per image at shape (1, 3, 320, 640)

0: 384x640 1 person, 213.0ms
Speed: 17.1ms preprocess, 213.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 448x640 1 person, 224.1ms
Speed: 5.5ms preprocess, 224.1ms inference, 3.2ms postprocess per image at shape (1, 3, 448, 640)

0: 640x640 1 person, 296.0ms
Speed: 8.6ms preprocess, 296.0ms inference, 2.0ms postprocess pe

In [40]:
train_data_3d = []
count = 0
for i in train_data:
    if(len(i[0][0]) == 17):
        train_data_3d.append(i[0][0]) 
    else: print(count)
    count+=1
trained_data = np.array(train_data_3d)
len(trained_data)


17
34
48
50
71
86
117
118
130
145
171
181
185
197
328
335
344
517
618


1048

In [65]:
test_data = process_dataset(r"D:\archith\DATASET\TEST")

test_data_3d = []
count = 0
for i in test_data:
    if(len(i[0][0]) == 17):
        test_data_3d.append(i[0][0]) 
    else: print(count)
    count+=1
test_data = np.array(test_data_3d)





0: 384x640 1 person, 318.3ms
Speed: 247.9ms preprocess, 318.3ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 211.7ms
Speed: 2.4ms preprocess, 211.7ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 352x640 1 person, 245.8ms
Speed: 3.4ms preprocess, 245.8ms inference, 2.6ms postprocess per image at shape (1, 3, 352, 640)

0: 448x640 1 person, 244.6ms
Speed: 4.4ms preprocess, 244.6ms inference, 2.8ms postprocess per image at shape (1, 3, 448, 640)

0: 384x640 1 person, 215.3ms
Speed: 2.1ms preprocess, 215.3ms inference, 11.1ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 1 person, 454.4ms
Speed: 2.0ms preprocess, 454.4ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

0: 416x640 1 person, 258.3ms
Speed: 3.3ms preprocess, 258.3ms inference, 4.0ms postprocess per image at shape (1, 3, 416, 640)

0: 384x640 1 person, 232.1ms
Speed: 2.1ms preprocess, 232.1ms inference, 3.3ms postprocess per image

In [66]:
label1 = [1]*208
label2 = [2]*177
label3 = [3]*263
label4 = [4]*160
label5 = [5]*240
label = label1+label2+label3+label4+label5
labels = np.array(label)

In [67]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Define the neural network
model = Sequential()
model.add(Flatten(input_shape=(17, 2)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))  # Adjust the number of output units to match your number of classes

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(trained_data, labels, epochs=800, batch_size=1048)

Epoch 1/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.1107 - loss: 2.2518
Epoch 2/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.1603 - loss: 2.2190
Epoch 3/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.1994 - loss: 2.1879
Epoch 4/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.2147 - loss: 2.1582
Epoch 5/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2271 - loss: 2.1298
Epoch 6/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.2281 - loss: 2.1031
Epoch 7/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.2309 - loss: 2.0780
Epoch 8/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2328 - loss: 2.0545
Epoch 9/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.2290 - loss: 2.0320
Epoch 10/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.2300 - loss: 2.0103
Epoch 11/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.2300 - loss: 1.9891
Epoch 12/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.2328 - loss

In [68]:
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)
for i in predicted_classes:
    print(i)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
3
3
3
3
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
4
2
2
2
5
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
5
2
2
2
2
2
2
5
2
2
2
2
2
2
2
2
5
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
5
2
2
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
2
1
1
3
1
1
1
1
1
1
1
1
1
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
2
5
5
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
2
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
5
5
5
5
5
5
